In [1]:
import dask.distributed
lc = dask.distributed.LocalCluster(n_workers=8, processes=True, memory_limit=200000000000)
client = dask.distributed.Client(lc)

In [1]:
from PyContact.core.ContactManager import ContactManager
from PyContact.core.ContactAnalyzer import ContactAnalyzer
topo = "/mnt/drive/workspace/pycontactData/str_biot/nowater.psf"
trajs = ["/mnt/drive/workspace/pycontactData/str_biot/trajectory-1.dcd"]
# topo = "PyContact/exampleData/rpn11_ubq.psf"
# trajs = ["PyContact/exampleData/rpn11_ubq.dcd"]
man = ContactManager(topo, trajs, 5.0, 2.5, 120, "segid STRA", "segid STRB")
%time man.readTrajectories(nproc=8, use_pmda=False)

starting analysis with multiprocessing


/home/max/anaconda3/lib/python3.6/site-packages/MDAnalysis/core/__init__.py:261: DeprecationWarning: MDAnalysis.core.flags is deprecated and will be removed in version 1.0. Use periodic=True/False to select_atoms
  DeprecationWarning)
/home/max/anaconda3/lib/python3.6/site-packages/MDAnalysis/core/__init__.py:261: DeprecationWarning: MDAnalysis.core.flags is deprecated and will be removed in version 1.0. This flag is obsolete, all selections now automatically select the fastest method
  DeprecationWarning)


CPU times: user 1min 35s, sys: 7.99 s, total: 1min 43s
Wall time: 3min 42s


In [4]:
def summarize(man):
    import pandas as pd
    import numpy as np
    cs = man.atomicContactTrajectories[0].contacts
    cs_r = np.concatenate(cs)
    df = pd.DataFrame(cs_r, columns=["frame", "idx1", "idx2", "weight", "hbonds"])

    df["idx1"] = np.asarray(df["idx1"], dtype=np.int)
    df["idx2"] = np.asarray(df["idx2"], dtype=np.int)
    df["frame"] = np.asarray(df["frame"], dtype=np.int)
    df["hbonds"] = np.asarray(df["hbonds"], dtype=np.int)

    df["resname1"] = man.atomicContactTrajectories[0].resname_array[df["idx1"]]
    df["resname2"] = man.atomicContactTrajectories[0].resname_array[df["idx2"]]

    df["resid1"] = man.atomicContactTrajectories[0].resid_array[df["idx1"]]
    df["resid2"] = man.atomicContactTrajectories[0].resid_array[df["idx2"]]
    # df["accumulatePattern"] = df["resname1"] + df["resid1"].astype(str) + df["resname2"] + df["resid2"].astype(str)

    df2 = df.groupby(['frame', 'resname1', 'resid1', 'resname2', 'resid2'])['weight'].agg('sum').reset_index()
    return df2

%time summary = summarize(man)
display(summary)

8.93 s ± 88.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


,frame,resname1,resid1,resname2,resid2,weight
0,0,ALA,63,LYS,80,0.482907
1,0,ALA,72,ALA,89,1.713780
2,0,ALA,72,GLY,113,0.059339
3,0,ALA,72,HSD,87,0.020611
4,0,ALA,72,SER,88,1.911444
5,0,ALA,72,THR,111,1.192155
6,0,ALA,78,ARG,59,1.429627
7,0,ALA,78,ASP,61,0.731899
8,0,ALA,78,TYR,60,1.805580
9,0,ALA,89,ALA,72,0.269642


In [2]:
%time man.accumulateContacts(map1=[0,0,1,1,0], map2=[0,0,1,1,0])

CPU times: user 10min 12s, sys: 750 ms, total: 10min 12s
Wall time: 10min 13s


In [ ]:
# test filters
from PyContact.core.ContactFilters import get_contacts_of_type, get_subrange, get_sorted_contact_scores
import matplotlib.pyplot as plt

ctrj = man.accumulatedContactTrajectories[0]

# cf = get_subrange(ctrj.contactScores, [25, 44])

mask = get_contacts_of_type(ctrj.contactTypes, "hbonds")
hbonds_only = ctrj.contactScores[mask[0],:]
plt.imshow(hbonds_only)
print(mask)

In [ ]:
mask2 = get_sorted_contact_scores(hbonds_only, u"median")
plt.imshow(hbonds_only[mask2][::-1])